In [ ]:
#!pip install geojson

In [ ]:
#pip install --upgrade pip

In [ ]:
#!pip install gdal==3.11.3

In [6]:
#geojson_path = "/Users/joana/git/gmcp/gmcp/bike_lane.geojson"
#geojson_path = "/Users/joana/git/gmcp/gmcp/bike_lane_fail_RFC.geojson"
#geojson_path = "/Users/joana/git/gmcp/gmcp/bike_lane_fail_schema.geojson"
geojson_path = "/Users/joana/git/gmcp/gmcp/bike_lane fail_geom.geojson"

In [7]:
# Validate geometries using GDAL

In [8]:
import sys
from osgeo import gdal, ogr

gdal.UseExceptions()

#print(f"GDAL Version: {gdal.__version__}")

In [9]:
def validate_geometries(file_path):
    print(f"Reading file: {file_path}")
    
    try:
        # Open the dataset using gdal.OpenEx (more robust than ogr.Open)
        ds = gdal.OpenEx(file_path, gdal.OF_VECTOR)
        
        if ds is None:
            print("Failed to open file. Please check the path and file integrity.")
            return


        layer = ds.GetLayer()
        layer_name = layer.GetName()
        feature_count = layer.GetFeatureCount()
        
        print(f"Layer '{layer_name}' loaded successfully.")
        print(f"Total features: {feature_count}")

        invalid_count = 0
        
        # Iterate over features
        # Note: We use a loop that ensures the feature stays in scope while we check it
        for feature in layer:
            fid = feature.GetFID()
            geom = feature.GetGeometryRef()
            
            # Check if geometry exists (some features might be null geometry)
            if geom is None:
                print(f"Feature ID {fid}: No Geometry found (NULL).")
                invalid_count += 1
                continue
                
            # Check validity
            if not geom.IsValid():
                # In standard OGR Python, IsValid() returns False but doesn't always 
                # give the specific reason string easily.
                print(f"Feature ID {fid}: INVALID geometry.")
                invalid_count += 1
        
        if invalid_count == 0:
            print("Validation Complete: All geometries are valid.")
        else:
            print(f"Validation Complete: Found {invalid_count} invalid features.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [10]:
validate_geometries(geojson_path)

Reading file: /Users/joana/git/gmcp/gmcp/bike_lane fail_geom.geojson
Layer 'bike_lane' loaded successfully.
Total features: 8
Feature ID 1: No Geometry found (NULL).
Validation Complete: Found 1 invalid features.


In [11]:
# Validate GeoJSON structure against RFC

In [12]:
import geojson

In [13]:
def validate_structure(file_path):
    print(f"Reading file: {file_path}")
    
    try:
        with open(file_path, 'r') as f:
            obj = geojson.load(f)

        if not isinstance(obj, geojson.GeoJSON):
            print("ERROR: File is a valid JSON, but is not a valid GeoJSON object.")
            return

        print(f"Detected Type: {type(obj).__name__}")
        return obj

    except json.JSONDecodeError as e:
        print("CRITICAL ERROR: The file is not valid JSON syntax.")
        print(f"Details: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [14]:
geojson_data = validate_structure(geojson_path)

Reading file: /Users/joana/git/gmcp/gmcp/bike_lane fail_geom.geojson
Detected Type: FeatureCollection


In [15]:
# Validate against JSON schema

In [16]:
#!pip install jsonschema

In [17]:
import json
from jsonschema import validate, ValidationError

In [18]:
schema_path = '/Users/joana/git/gmcp/gmcp/bike-schema.json'

In [19]:
#geojson_data

In [20]:
try:
    with open(schema_path, 'r') as f:
        schema_data = json.load(f)
    print(f"Loaded schema: {schema_path}")
except FileNotFoundError:
    print(f"Error: Could not find {schema_path}")

Loaded schema: /Users/joana/git/gmcp/gmcp/bike-schema.json


In [21]:
try:
    # This function compares the data against the schema
    validate(instance=geojson_data, schema=schema_data)
    print("SUCCESS: The GeoJSON file is valid according to the schema.")
    
except ValidationError as e:
    print("FAILURE: The GeoJSON file is INVALID.")
    print(f"Error message: {e.message}")
    print(f"Error location: {list(e.path)}")
    
except NameError:
    print("Files were not loaded correctly, cannot validate.")

FAILURE: The GeoJSON file is INVALID.
Error message: None is not of type 'object'
Error location: ['features', 0, 'geometry']
